## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [37]:
import pandas as pd

import numpy as np

df = pd.read_csv('client_dataset.csv')
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [8]:
# View the column names in the data
df.columns

Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [7]:
# Use the describe function to gather some basic statistics
df.describe()

,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


In [11]:
# Use this space to do any additional research
# and familiarize yourself with the data.
unique_jobs = df['job'].nunique()
print(unique_jobs)

498


In [17]:
# What three item categories had the most entries?
category_counts = df['category'].value_counts().sort_values(ascending=False)
#print(category_counts)
category_counts.head(3)

category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [22]:
# For the category with the most entries, which subcategory had the most entries?
df_consumables = df[df['category']=='consumables']
# print(df_consumables.head())

subcategory_counts = df_consumables['subcategory'].value_counts().sort_values(ascending=False)
# print(subcategory_counts)

# top_consumable_subcategory = subcategory_counts.max()

# print(top_consumable_subcategory)

subcategory_counts.head(1)

subcategory
bathroom supplies    6424
Name: count, dtype: int64

In [26]:
# Which five clients had the most entries in the data?
client_counts = df['client_id'].value_counts().sort_values(ascending=False)
#print(category_counts)
client_counts.head(5)

client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [25]:
# Store the client ids of those top 5 clients in a list.
top_5_clients = client_counts.head(5).index.tolist()
print(top_5_clients)

[33615, 66037, 46820, 38378, 24741]


In [29]:
# How many total units (the qty column) did the client with the most entries order order?
top_client = df[df['client_id'] == 33615]

top_client_units = top_client['qty'].sum()

print(top_client_units)

64313


## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [65]:
# Create a column that calculates the subtotal for each line using the unit_price and the qty
df['subtotal'] = df['unit_price'] * df['qty']
# print(df.head())

compare_clients = df[df['client_id'].isin([66037,33615])]
compare_clients_price = compare_clients.groupby('client_id')['subtotal'].sum()
print(compare_clients_price)

client_id
33615    5839032.11
66037    7995708.38
Name: subtotal, dtype: float64


In [71]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.

df['total_weight'] = df['unit_weight'] * df['qty']

def calculate_shipping_price(total_weight):
    if total_weight > 50:
        return total_weight * 7
    else:
        return total_weight * 10

df['shipping_price'] = df['total_weight'].apply(calculate_shipping_price)

print(df.head())


       first      last                    job               phone  \
0     Donald   Harding           Immunologist  793-904-7725x39308   
1    Tiffany     Myers        Music therapist    201.442.4543x942   
2    Shannon    Watson           Immunologist   687.737.9424x8503   
3     Nathan     Baker  Accounting technician    827-788-8123x012   
4  Christina  Schwartz           Chiropractor        265-829-3643   

                                 email  client_id  order_id  order_date  \
0     harding.donald.7185@sullivan.com      58515   8953482  2023-04-28   
1    myers.t.6537@ferguson-johnson.net      37609   8069089  2023-05-19   
2                swatson8146@payne.net      57113   1902144  2023-01-29   
3               bakernathan@benson.com      46554   9031802  2023-04-25   
4  christinaschwartz9252@mcconnell.com      92089   1322274  2023-05-28   

   order_week  order_year  ... unit_cost unit_weight  qty  line_number  \
0          17        2023  ...    762.71        7.50  105   

In [72]:
# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%

# df.rename(columns={'total_price': 'subtotal'}, inplace=True)

df['total_price'] = (df['subtotal'] + df['shipping_price']) * 1.0925

# print(df.head())

print(df[['subtotal', 'shipping_price', 'total_price']].head())

    subtotal  shipping_price    total_price
0  115164.00         5512.50  131839.076250
1     523.95          312.90     914.258625
2     527.28          458.64    1077.117600
3    1056.18          356.70    1543.571400
4    3902.00         6500.20   11364.403500


In [73]:
# Create a column for the cost of each line using unit cost, qty, and
# shipping price (assume the shipping cost is exactly what is charged to the client).
df['line_cost'] = df['unit_cost'] * df['qty'] + df['shipping_price']

print(df.head(5))


       first      last                    job               phone  \
0     Donald   Harding           Immunologist  793-904-7725x39308   
1    Tiffany     Myers        Music therapist    201.442.4543x942   
2    Shannon    Watson           Immunologist   687.737.9424x8503   
3     Nathan     Baker  Accounting technician    827-788-8123x012   
4  Christina  Schwartz           Chiropractor        265-829-3643   

                                 email  client_id  order_id  order_date  \
0     harding.donald.7185@sullivan.com      58515   8953482  2023-04-28   
1    myers.t.6537@ferguson-johnson.net      37609   8069089  2023-05-19   
2                swatson8146@payne.net      57113   1902144  2023-01-29   
3               bakernathan@benson.com      46554   9031802  2023-04-25   
4  christinaschwartz9252@mcconnell.com      92089   1322274  2023-05-28   

   order_week  order_year  ... unit_cost unit_weight  qty  line_number  \
0          17        2023  ...    762.71        7.50  105   

In [74]:
# Create a column for the profit of each line using line cost and line price
df['profit'] = df['total_price'] - df['line_cost']

print(df[['total_price', 'line_cost', 'profit']].head())


     total_price  line_cost        profit
0  131839.076250   85597.05  46242.026250
1     914.258625     629.79    284.468625
2    1077.117600     765.18    311.937600
3    1543.571400    1077.35    466.221400
4   11364.403500    8663.60   2700.803500


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [76]:
# Check your work using the totals above
compare_orders = df[df['order_id'].isin([2742071,2173913,6128929])]
compare_orders_price = compare_orders.groupby('order_id')['total_price'].sum()
print(compare_orders_price)


# check_order = df[df['order_id'].isin([2742071])]
# print(check_order)


order_id
2173913    162388.719300
2742071    152811.897075
6128929    923441.240450
Name: total_price, dtype: float64


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [84]:
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.
top_5_spending = df[df['client_id'].isin(top_5_clients)].groupby('client_id')['total_price'].sum()
print(top_5_spending)

client_id
24741    8.226889e+07
33615    8.377309e+06
38378    1.290655e+07
46820    9.743794e+06
66037    1.025951e+07
Name: total_price, dtype: float64


In [110]:
# Create a summary DataFrame showing the totals for the for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit. 
top_5_summary = df[df['client_id'].isin(top_5_clients)].groupby('client_id').agg({'qty':'sum','shipping_price':'sum','total_price':'sum','line_cost':'sum','profit':'sum'})
print(top_5_summary)


              qty  shipping_price   total_price    line_cost        profit
client_id                                                                 
24741      239862      5126448.37  8.226889e+07  45688899.71  3.657999e+07
33615       64313      1828984.89  8.377309e+06   6175313.91  2.201995e+06
38378       73667      3429455.40  1.290655e+07   9634720.98  3.271830e+06
46820       75768      1601448.84  9.743794e+06   7007191.64  2.736603e+06
66037       43018      1395151.85  1.025951e+07   7004482.98  3.255032e+06


In [111]:
# Format the data and rename the columns to names suitable for presentation.

# Define the money columns. 
money_columns = ['shipping_price', 'total_price', 'line_cost', 'profit']

# Define a function that converts a dollar amount to millions.
def convert_to_millions(x):
    return f"${x/1_000_000:,.2f}"

# Apply the currency_format_millions function to only the money columns. 
top_5_summary[money_columns] = top_5_summary[money_columns].applymap(convert_to_millions)


# Rename the columns to reflect the change in the money format. 
top_5_summary.rename(columns={
    'shipping_price': 'Total Shipping Price (Millions)',
    'total_price': 'Total Revenue (Millions)',
    'line_cost': 'Total Line Cost (Millions)',
    'profit': 'Total Profit (Millions)'
}, inplace=True)

print(top_5_summary)



              qty Total Shipping Price (Millions) Total Revenue (Millions)  \
client_id                                                                    
24741      239862                           $5.13                   $82.27   
33615       64313                           $1.83                    $8.38   
38378       73667                           $3.43                   $12.91   
46820       75768                           $1.60                    $9.74   
66037       43018                           $1.40                   $10.26   

          Total Line Cost (Millions) Total Profit (Millions)  
client_id                                                     
24741                         $45.69                  $36.58  
33615                          $6.18                   $2.20  
38378                          $9.63                   $3.27  
46820                          $7.01                   $2.74  
66037                          $7.00                   $3.26  


C:\Users\Jesse Poljak\AppData\Local\Temp\ipykernel_21824\162305421.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  top_5_summary[money_columns] = top_5_summary[money_columns].applymap(convert_to_millions)


In [112]:
# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.
top_5_sorted = top_5_summary.sort_values(by='Total Profit (Millions)', ascending = False)
print(top_5_sorted)

              qty Total Shipping Price (Millions) Total Revenue (Millions)  \
client_id                                                                    
24741      239862                           $5.13                   $82.27   
38378       73667                           $3.43                   $12.91   
66037       43018                           $1.40                   $10.26   
46820       75768                           $1.60                    $9.74   
33615       64313                           $1.83                    $8.38   

          Total Line Cost (Millions) Total Profit (Millions)  
client_id                                                     
24741                         $45.69                  $36.58  
38378                          $9.63                   $3.27  
66037                          $7.00                   $3.26  
46820                          $7.01                   $2.74  
33615                          $6.18                   $2.20  


In [ ]:
#Summary
#Largely due to the about 3 times higher quantity than the next largest client, client 24741 had by far the highest revenue and profit. Beyond that, they even have the highest profit margin, which is what really helps them blow the other clients out of the water at $37 million in profit. The next largest clients hover around $2-4 million in profit.